# Kiểm tra GPU

In [1]:
!nvidia-smi

Tue May 14 15:43:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Cài đặt và import các thư viện cần thiết

In [2]:
!pip install transformers datasets

In [3]:
pip install accelerate -U

# Restart runtime rồi chạy code phía dưới

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, TextClassificationPipeline
from datasets import load_dataset, load_from_disk, load_metric
import numpy as np

# Load model và tokenizer

In [5]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer

In [6]:
model = BertForSequenceClassification.from_pretrained("trituenhantaoio/bert-base-vietnamese-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained("trituenhantaoio/bert-base-vietnamese-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at trituenhantaoio/bert-base-vietnamese-uncased and are newly initialized: ['classifier.bias', 'classifi

In [7]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# Dataset

In [8]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd

df1_train = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/train_victsd.csv')
df1_valid = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/dev_victsd.csv')
df1_test = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/test_victsd.csv')

df2_train = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/train_vihsd.csv')
df2_valid = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/dev_vihsd.csv')
df2_test = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/test_vihsd.csv')

# # Nối các DataFrame theo chiều dọc (axis=0)
# concatenated_df1 = pd.concat([df1a, df1b], axis=0)

# # Nếu bạn muốn đặt lại chỉ mục của DataFrame sau khi nối
# concatenated_df1.reset_index(drop=True, inplace=True)

# # Nối các DataFrame theo chiều dọc (axis=0)
# concatenated_df2 = pd.concat([df2a, df2b], axis=0)

# # Nếu bạn muốn đặt lại chỉ mục của DataFrame sau khi nối
# concatenated_df2.reset_index(drop=True, inplace=True)

In [10]:
df2_train.head()

,text,label
0,Em được làm fan cứng luôn rồi nè ❤️ reaction q...,0
1,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,2
2,Đậu Văn Cường giờ giống thằng sida hơn à,0
3,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN. NHÀ NƯ...,2
4,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,0


In [11]:
def process_df2(df):
    df['label'] = df['label'].replace(2, 1)
    return df.dropna()

df2_train = process_df2(df2_train)
df2_valid = process_df2(df2_valid)
df2_test = process_df2(df2_test)

In [12]:
df2_train.isna().sum()

text     0
label    0
dtype: int64

In [13]:
def process_df1(df):
    return df.dropna()
df1_train = process_df1(df1_train)
df1_valid = process_df1(df1_valid)
df1_test = process_df1(df1_test)

In [14]:
df1_valid.isna().sum()

text     0
label    0
dtype: int64

In [15]:
tu_dien_viet_tat = {
    "a": "anh",
    "c": "chị",
    "e": "em",
    "b": "bạn",
    "bn": "bạn",
    "t": "tao",
    "m": "mày",
    "cmt": "bình luận",
    "ng": "người",
    "mn": "mọi người",
    "iu": "yêu",
    "ct": "chia tay",
    "ny": "người yêu",
    "vk": "vợ",
    "ck": "chồng",
    "vc": "vợ chồng",
    "ox": "ông xã",
    "bx": "bà xã",
    "h": "giờ",
    "g": "giờ",
    "ph": "phút",
    "bh": "bây giờ",
    "bjo": "bây giờ",
    "hn": "hôm nay",
    "hnay": "hôm nay",
    "hqua": "hôm qua",
    "htrc": "hôm trước",
    "nt": "nhắn tin",
    "nc": "nói chuyện",
    "mún": "muốn",
    "bit": "biết",
    "bik": "biết",
    "cb": "chuẩn bị",
    "nch": "nói chung",
    "dt": "điện thoại",
    "đt": "điện thoại",
    "bth": "bình thường",
    "j": "gì",
    "bn": "bao nhiêu",
    "ntn": "như thế nào",
    "k": "không",
    "ko": "không",
    "hok": "không",
    "hem": "không",
    "dc": "được",
    "đc": "được",
    "r": "rồi",
    "rùi": "rồi",
    "nhìu": "nhiều",
    "wá": "quá",
    "cg": "cũng",
    "uk": "ừ",
    "uh": "ừ",
    "ukm": "ừ",
    "G9": "ngủ ngon",
    "hic": "tiếng khóc",
    "vn": "Việt Nam"
}

In [16]:
!pip install emoji_vietnamese

In [17]:
!pip install underthesea

In [18]:
import underthesea
import string
import emoji_vietnamese

def preprocess_data(data):
    data = data.replace(
        r'http\S+', '').replace(r'www\S+', '')
    data = data.replace(
        '['+string.punctuation+']', '')
    data = data.lower().strip()
    data = emoji_vietnamese.demojize(data)
    new_str = data.split()
    for i in range(len(new_str)):
       if new_str[i] in tu_dien_viet_tat.keys():
         new_str[i] = tu_dien_viet_tat[new_str[i]]
    return underthesea.word_tokenize(" ".join(new_str), format='text')


In [19]:
df1_train['text'] = df1_train['text'].map(preprocess_data)
df1_valid['text'] = df1_valid['text'].map(preprocess_data)
df1_test['text'] = df1_test['text'].map(preprocess_data)

df2_train['text'] = df2_train['text'].map(preprocess_data)
df2_valid['text'] = df2_valid['text'].map(preprocess_data)
df2_test['text'] = df2_test['text'].map(preprocess_data)

In [20]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42, sampling_strategy=1)
X_resampled, y_resampled = ros.fit_resample(df1_train[['text']], df1_train['label'])
df1_train_new = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)
df1_train_new.columns = ['text', 'label']

X_resampled, y_resampled = ros.fit_resample(df2_train[['text']], df2_train['label'])
df2_train_new = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)
df2_train_new.columns = ['text', 'label']

In [21]:
from datasets import Dataset

def convert_to_datasets(df):
    dataset = Dataset.from_pandas(df).shuffle(seed=20)
    return dataset

train_dataset1 = convert_to_datasets(df1_train_new)
valid_dataset1 = convert_to_datasets(df1_valid)
test_dataset1 = convert_to_datasets(df1_test)

train_dataset2 = convert_to_datasets(df2_train_new)
valid_dataset2 = convert_to_datasets(df2_valid)
test_dataset2 = convert_to_datasets(df2_test)

In [22]:
train_dataset1

Dataset({
    features: ['text', 'label'],
    num_rows: 12482
})

In [23]:
# Tokenizer dataset và thêm padding
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=256)
# tokenized_train_dataset1 = train_dataset1.map(tokenize_function, batched=True, remove_columns='text')
# tokenized_valid_dataset1 = valid_dataset1.map(tokenize_function, batched=True, remove_columns='text')
# tokenized_test_dataset1 = test_dataset1.map(tokenize_function, batched=True, remove_columns='text')

tokenized_train_dataset2 = train_dataset2.map(tokenize_function, batched=True, remove_columns='text')
tokenized_valid_dataset2 = valid_dataset2.map(tokenize_function, batched=True, remove_columns='text')
tokenized_test_dataset2 = test_dataset2.map(tokenize_function, batched=True, remove_columns='text')
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/12482 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/37164 [00:00<?, ? examples/s]

Map:   0%|          | 0/2672 [00:00<?, ? examples/s]

Map:   0%|          | 0/6680 [00:00<?, ? examples/s]

# Training

In [24]:
# Viết hàm tính accuracy
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-24-b113ca1a6718>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [25]:
!nvidia-smi

Tue May 14 15:47:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [26]:
# Cài đặt các tham số và train
training_args = TrainingArguments(
    output_dir="./results1",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

trainer_1 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset1,
    eval_dataset=tokenized_valid_dataset1,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # data_collator=data_collator,
)



In [27]:
trainer_1.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.300900,0.586822,0.886000


TrainOutput(global_step=1561, training_loss=0.3008872697477078, metrics={'train_runtime': 611.9867, 'train_samples_per_second': 20.396, 'train_steps_per_second': 2.551, 'total_flos': 1642076096501760.0, 'train_loss': 0.3008872697477078, 'epoch': 1.0})

In [37]:

from sklearn.metrics import precision_recall_fscore_support
import numpy as np

def compute_metrics_f1(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # precision_recall_fscore_support returns a tuple containing precision, recall, f1, support respectively
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None)

    # Create a dictionary to store metrics for each class
    metrics = {}
    for i, (p, r, f) in enumerate(zip(precision, recall, f1)):
        metrics[f'label_{i}'] = {
            'precision': p,
            'recall': r,
            'f1': f,
        }
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    metrics[f'all'] = {
            'precision': precision,
            'recall': recall,
            'f1': f1,
            }

    return metrics


trainer_eval = Trainer(
    model=model,
    eval_dataset=tokenized_test_dataset1,
    compute_metrics=compute_metrics_f1,

)

trainer_eval.evaluate()


Trainer is attempting to log a value of "{'precision': 0.8951048951048951, 'recall': 0.7191011235955056, 'f1': 0.7975077881619939}" of type <class 'dict'> for key "eval/label_0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.12280701754385964, 'recall': 0.3181818181818182, 'f1': 0.17721518987341772}" of type <class 'dict'> for key "eval/label_1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.12280701754385964, 'recall': 0.3181818181818182, 'f1': 0.17721518987341772}" of type <class 'dict'> for key "eval/all" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.7276970744132996,
 'eval_label_0': {'precision': 0.8951048951048951,
  'recall': 0.7191011235955056,
  'f1': 0.7975077881619939},
 'eval_label_1': {'precision': 0.12280701754385964,
  'recall': 0.3181818181818182,
  'f1': 0.17721518987341772},
 'eval_all': {'precision': 0.12280701754385964,
  'recall': 0.3181818181818182,
  'f1': 0.17721518987341772},
 'eval_runtime': 16.5393,
 'eval_samples_per_second': 60.462,
 'eval_steps_per_second': 7.558}

In [32]:
model = AutoModelForSequenceClassification.from_pretrained("FPTAI/vibert-base-cased", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at FPTAI/vibert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
# Cài đặt các tham số và train
training_args = TrainingArguments(
    output_dir="./results2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

trainer_2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset2,
    eval_dataset=tokenized_valid_dataset2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # data_collator=data_collator,
)

trainer_2.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.426300,0.411392,0.845060


Epoch,Training Loss,Validation Loss,Accuracy
1,0.426300,0.411392,0.845060
2,0.204500,0.678057,0.850299
3,0.099000,0.773129,0.861901


TrainOutput(global_step=6969, training_loss=0.24323628612118176, metrics={'train_runtime': 4813.5757, 'train_samples_per_second': 23.162, 'train_steps_per_second': 1.448, 'total_flos': 1.466738889209856e+16, 'train_loss': 0.24323628612118176, 'epoch': 3.0})

In [38]:
trainer_eval = Trainer(
    model=model,
    eval_dataset=tokenized_test_dataset2,
    compute_metrics=compute_metrics_f1,

)

trainer_eval.evaluate()

Trainer is attempting to log a value of "{'precision': 0.9277451166318984, 'recall': 0.8817591925018025, 'f1': 0.9041678218279272}" of type <class 'dict'> for key "eval/label_0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.5337597725657427, 'recall': 0.6634275618374559, 'f1': 0.5915714848365499}" of type <class 'dict'> for key "eval/label_1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.5337597725657427, 'recall': 0.6634275618374559, 'f1': 0.5915714848365499}" of type <class 'dict'> for key "eval/all" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.4033377170562744,
 'eval_label_0': {'precision': 0.9277451166318984,
  'recall': 0.8817591925018025,
  'f1': 0.9041678218279272},
 'eval_label_1': {'precision': 0.5337597725657427,
  'recall': 0.6634275618374559,
  'f1': 0.5915714848365499},
 'eval_all': {'precision': 0.5337597725657427,
  'recall': 0.6634275618374559,
  'f1': 0.5915714848365499},
 'eval_runtime': 106.2839,
 'eval_samples_per_second': 62.851,
 'eval_steps_per_second': 7.856}

# Predict

In [41]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
pipe("Phim này hay vãi")

[[{'label': 'LABEL_0', 'score': 0.9850435256958008},
  {'label': 'LABEL_1', 'score': 0.014956467784941196}]]

In [42]:
model.num_parameters()

115355906